# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [2]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')
data.info()
print(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4  

**Вывод**

В исходной таблице имеем 12 столбцов, а также информацию о 21525 людях, ни один столбец не является полностью пустым, поэтому приступаем к дальшейшей обработке.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [5]:
df = data.copy()

df.education = df.education.str.lower()
df.pivot_table(index='income_type', columns='education', values='total_income', aggfunc='median')

education,высшее,начальное,неоконченное высшее,среднее,ученая степень
income_type,,,,,
безработный,202722.511368,NaN,NaN,59956.991984,NaN
в декрете,NaN,NaN,NaN,53829.130729,NaN
госслужащий,157982.545567,145017.937533,156266.846399,144351.310834,111392.231107
компаньон,190976.633414,140908.421338,169674.017186,149093.468381,NaN
пенсионер,145017.937533,107398.699119,129742.715263,125440.020719,177088.845999
предприниматель,322090.541240,NaN,NaN,NaN,NaN
сотрудник,155103.948747,131629.331952,145017.937533,145017.937533,198570.757322
студент,98201.625314,NaN,NaN,NaN,NaN


In [7]:
df.pivot_table(index=['income_type', 'gender'], columns='education', values='total_income', aggfunc='median')

education                      высшее      начальное  неоконченное высшее  \
income_type     gender                                                      
безработный     F       202722.511368            NaN                  NaN   
                M                 NaN            NaN                  NaN   
в декрете       F                 NaN            NaN                  NaN   
госслужащий     F       145017.937533  125392.170417        145017.937533   
                M       204717.884325  190966.659534        166424.838494   
компаньон       F       174369.537430  139514.970550        164996.813904   
                M       217768.009290  150100.960964        179370.487646   
                XNA               NaN            NaN        203905.157261   
пенсионер       F       145017.937533  102598.653164        130837.388717   
                M       150246.754511  114068.787524        124667.471301   
предприниматель F       499163.144947            NaN                  NaN   
                M       145017.937533            NaN                  NaN   
сотрудник       F       145017.937533  124397.467932        143663.050521   
                M       179564.658485  145017.937533        175068.921286   
студент         M        98201.625314            NaN                  NaN   

education                     среднее  ученая степень  
income_type     gender                                 
безработный     F                 NaN             NaN  
                M        59956.991984             NaN  
в декрете       F        53829.130729             NaN  
госслужащий     F       135073.137857   111392.231107  
                M       160213.467715             NaN  
компаньон       F       145017.937533             NaN  
                M       170206.988773             NaN  
                XNA               NaN             NaN  
пенсионер       F       123508.906819   255425.196556  
                M       134708.033691    98752.495442  
предприниматель F                 NaN             NaN  
                M                 NaN             NaN  
сотрудник       F       132635.207938             NaN  
                M       147530.985003   198570.757322  
студент         M                 NaN             NaN

In [8]:
qq = df.pivot_table(index=['income_type', 'gender'], columns='education', values='total_income', aggfunc='median')
qq.loc[('студент', 'M')]['высшее']

98201.62531401133

In [9]:
def super_fillna_func(income_type, gender, education):
    '''
    Находит в таблице qq нужную медиану.
    '''
    try:
        return qq.loc[(income_type, gender)][education]
    except:
        return 0
    
print(super_fillna_func('студент', 'M','высшее') ) 

98201.62531401133


In [10]:
print(super_fillna_func('ревьюер', 'F','высшее') )

0


In [25]:
df.apply(lambda row: super_fillna_func(row['income_type'], row['gender'], row['education']), axis=1)

0        145017.937533
1        132635.207938
2        147530.985003
3        147530.985003
4        123508.906819
             ...      
21520    145017.937533
21521    123508.906819
21522    147530.985003
21523    147530.985003
21524    132635.207938
Length: 21525, dtype: float64

In [26]:
data1 = data.pivot_table(index=['income_type', 'gender'], columns='education', values='total_income', aggfunc='median')
data1 = data1.apply(lambda x: x.fillna(x.median()), axis=1)
print(data1)

education                      ВЫСШЕЕ         Высшее      НАЧАЛЬНОЕ  \
income_type     gender                                                
безработный     F       202722.511368  202722.511368  202722.511368   
                M        59956.991984   59956.991984   59956.991984   
в декрете       F        53829.130729   53829.130729   53829.130729   
госслужащий     F       162361.426048  145017.937533  134974.938699   
                M       187365.789659  245357.105783  180482.467306   
компаньон       F       185492.137323  186150.322083  255696.018913   
                M       205309.210389  209878.152318  131455.738975   
                XNA     203905.157261  203905.157261  203905.157261   
пенсионер       F       164765.086709  145017.937533   70278.364094   
                M       155882.271260  362784.960182  104293.022877   
предприниматель F       499163.144947  499163.144947  499163.144947   
                M       145017.937533  145017.937533  145017.937533   
сотруд

In [27]:
# Запишем в новый столбец.
df['new_income'] = df.apply(lambda row: super_fillna_func(row['income_type'], row['gender'], row['education']), axis=1)

# Пандас сам заменит пропуски значениями из нового столбца в той же строке.
df['total_income'] = df['total_income'].fillna(df['new_income'])

**Вывод**

Найдены пропущенные значения во втором и одиннадцатом столбцах. Данные пропущенные значения являются числовыми значениями, появление пропусков могло было быть вызвано неправильным внесением данных в базу, а также могло быть так, что данные не были предоставлены банку в полном размере. Пропуски заполнены средними значениями по столбцу, то есть среднему значению по трудоустроенным дням и общему заработку.

### Замена типа данных

In [30]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Для изменения вещественного типа данных на целочисленных использовался метод astype(), потому что этот метод выполняет необходимые действия.

### Обработка дубликатов

In [31]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower() 
data['gender'] = data['gender'].str.lower() 
data['income_type'] = data['income_type'].str.lower() 
data['purpose'] = data['purpose'].str.lower() 
print(data.duplicated().sum())



71


**Вывод**

Были найдены 71 дубликат, которые были убраны с восстановлением индекса. Дубликаты могли появиться в следствие непрвильного внесения данных в таблицу, а также неправильной выгрузке данных 

### Лемматизация

In [33]:
from pymystem3 import Mystem
from collections import Counter

m = Mystem()

lemmas_list = []
for text in data['purpose']:
    lemmas = m.lemmatize(text)
    lemmas_list += lemmas

Counter(lemmas_list)

Counter({'покупка': 5912,
         ' ': 33677,
         'жилье': 4473,
         '\n': 21525,
         'приобретение': 462,
         'автомобиль': 4315,
         'дополнительный': 909,
         'образование': 4022,
         'сыграть': 774,
         'свадьба': 2348,
         'операция': 2610,
         'с': 2924,
         'на': 2233,
         'проведение': 777,
         'для': 1294,
         'семья': 641,
         'недвижимость': 6367,
         'коммерческий': 1315,
         'жилой': 1233,
         'строительство': 1881,
         'собственный': 635,
         'подержать': 479,
         'свой': 2235,
         'со': 630,
         'заниматься': 908,
         'сделка': 944,
         'подержанный': 489,
         'получение': 1316,
         'высокий': 1375,
         'профильный': 436,
         'сдача': 653,
         'ремонт': 612})

In [34]:
def purpose_change(purpose):
    lemmas_row = m.lemmatize(purpose)
    try:
        if 'жилье' in lemmas_row:
            return 'жилье'
        elif 'свадьба' in lemmas_row:
            return 'свадьба'
        elif 'образование' in lemmas_row:
            return 'образование'
        elif 'автомобиль' in lemmas_row:
            return 'автомобиль'
        elif 'недвижимость' in lemmas_row:
            return 'недвижимость'
        
        return 'другое'
    except:
        return 'ошибка'
    
# новый столбец purpose_new, туда сохраним результат работы функции
data['purpose_category'] = data['purpose'].apply(purpose_change)
print(data)

       children  days_employed  dob_years education  education_id  \
0             1          -8437         42    высшее             0   
1             1          -4024         36   среднее             1   
2             0          -5623         33   среднее             1   
3             3          -4124         32   среднее             1   
4             0         340266         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1          -4529         43   среднее             1   
21521         0         343937         67   среднее             1   
21522         1          -2113         38   среднее             1   
21523         3          -3112         38   среднее             1   
21524         2          -1984         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      f   сотрудник     0   
1       женат / замужем        

**Вывод**

Нашли самые ключевые слова, обозначающие цели взятия кредита с помощью подсчета лемм в столбце "цели"

### Категоризация данных

In [35]:
new_data_1 = data[['education', 'education_id', 'debt']]
new_data_1 = new_data_1.drop_duplicates().reset_index(drop=True)
print(new_data_1.head(15))

new_data_2 = data[['family_status', 'family_status_id', 'debt']]
new_data_2 = new_data_2.drop_duplicates().reset_index(drop=True)
print(new_data_2.head(15))

new_data_3 = data[['children', 'debt']]
new_data_3 = new_data_3.drop_duplicates().reset_index(drop=True)
print(new_data_3.head(20))

             education  education_id  debt
0               высшее             0     0
1              среднее             1     0
2  неоконченное высшее             2     0
3               высшее             0     1
4            начальное             3     0
5              среднее             1     1
6            начальное             3     1
7  неоконченное высшее             2     1
8       ученая степень             4     0
           family_status  family_status_id  debt
0        женат / замужем                 0     0
1       гражданский брак                 1     0
2       гражданский брак                 1     1
3         вдовец / вдова                 2     0
4              в разводе                 3     0
5  не женат / не замужем                 4     0
6  не женат / не замужем                 4     1
7        женат / замужем                 0     1
8         вдовец / вдова                 2     1
9              в разводе                 3     1
    children  debt
0          1

**Вывод**

<font color='blue'> Из данных видно, что образование, семейное положение и наличие детей не имеют прямой связи с возвращением долга </font>  

In [36]:
dict(zip(data['family_status_id'], data['family_status']))

{0: 'женат / замужем',
 1: 'гражданский брак',
 2: 'вдовец / вдова',
 3: 'в разводе',
 4: 'не женат / не замужем'}

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [37]:
df = data.groupby('children')['debt'].count()
data_grouped = data.groupby('children')['debt'].sum()

print('Процент невозврата в семьях с разным количеством детей:', (100*data_grouped/df))

Процент невозврата в семьях с разным количеством детей: children
-1      2.127660
 0      7.512898
 1      9.215442
 2      9.440389
 3      8.181818
 4      9.756098
 5      0.000000
 20    10.526316
Name: debt, dtype: float64


**Вывод**

<font color='blue'> Видим, что с большей вероятностью кредиторы, у которых есть дети, не будут выплачивать кредит в срок, в сравнении с кредиторами, у которых детей нет, но разница небольшая, поэтому в процентном соотношении можно сказать, что прямой и сильной зависимости не наблюдается </font> 

In [38]:
data.groupby('children').agg({'debt': ['count','sum','mean']})

debt                
          count   sum      mean
children                       
-1           47     1  0.021277
 0        14149  1063  0.075129
 1         4818   444  0.092154
 2         2055   194  0.094404
 3          330    27  0.081818
 4           41     4  0.097561
 5            9     0  0.000000
 20          76     8  0.105263

In [39]:
data.groupby('children')['debt'].agg(['count','sum','mean'])

,count,sum,mean
children,,,
-1,47,1,0.021277
0,14149,1063,0.075129
1,4818,444,0.092154
2,2055,194,0.094404
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000
20,76,8,0.105263


In [40]:
data.groupby('children')['debt'].agg(['count', 'sum', lambda x: x.mean()*100])

,count,sum,<lambda_0>
children,,,
-1,47,1,2.127660
0,14149,1063,7.512898
1,4818,444,9.215442
2,2055,194,9.440389
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000
20,76,8,10.526316


In [41]:
data.groupby('children')['debt'].agg(['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ])

,count,sum,<lambda_0>
children,,,
-1,47,1,2.13%
0,14149,1063,7.51%
1,4818,444,9.22%
2,2055,194,9.44%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.0%
20,76,8,10.53%


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [43]:
df1 = data.groupby('family_status')['debt'].count()
data_grouped1 = data.groupby('family_status')['debt'].sum()

print('Процент невозврата в семьях с разным семейным положением:', (100*data_grouped1/df1))

Процент невозврата в семьях с разным семейным положением: family_status
в разводе                7.112971
вдовец / вдова           6.562500
гражданский брак         9.288963
женат / замужем          7.520194
не женат / не замужем    9.740491
Name: debt, dtype: float64


**Вывод**

<font color='blue'> Видно, что люди в гражданском браке и холостые, имеют бОльшую предрасположенность к задержке выплаты кредита </font> 

In [44]:
print('Создали словарь:')
family_dict = data[['family_status_id', 'family_status']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
display(family_dict)


print('\n\nСгруппированная таблица. Берем по id, другой столбец удалили:')
a = data.groupby('family_status_id')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])
display(a)


# Заменяем
print('\n\nЗаменяем численные значения по ключу словаря:')
a.reset_index().replace({'family_status_id': family_dict.family_status.to_dict()})

Создали словарь:


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем




Сгруппированная таблица. Берем по id, другой столбец удалили:


,count,sum,<lambda_0>
family_status_id,,,
0,12380,931,7.52%
1,4177,388,9.29%
2,960,63,6.56%
3,1195,85,7.11%
4,2813,274,9.74%




Заменяем численные значения по ключу словаря:


,family_status_id,count,sum,<lambda_0>
0,женат / замужем,12380,931,7.52%
1,гражданский брак,4177,388,9.29%
2,вдовец / вдова,960,63,6.56%
3,в разводе,1195,85,7.11%
4,не женат / не замужем,2813,274,9.74%


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [45]:
data['income'] = pd.qcut(data['total_income'], 4, labels=["low", "pre-medium", "upper-medium", "high"])
df1 = data.groupby('income')['debt'].count()
data_grouped1 = data.groupby('income')['debt'].sum()

print('Процент невозврата в семьях с разным доходом:', 100*data_grouped1/df1)

Процент невозврата в семьях с разным доходом: income
low             7.933854
pre-medium      8.457019
upper-medium    8.872846
high            7.173388
Name: debt, dtype: float64


**Вывод**


<font color='blue'> Видно, что процент невозврата кредита в срок незначительно, но больше у людей со средним доходом </font>     

- Как разные цели кредита влияют на его возврат в срок?

In [46]:
df3 = data.groupby('purpose_category')['debt'].count()
data_grouped3 = data.groupby('purpose_category')['debt'].sum()

print('Процент невозврата в семьях с разными целями:', (100*data_grouped3/df3))

Процент невозврата в семьях с разными целями: purpose_category
автомобиль      9.339513
жилье           6.885759
недвижимость    7.444636
образование     9.199403
свадьба         7.921635
Name: debt, dtype: float64


**Вывод**

<font color='blue'> Видно, что реже всего в срок отдают кредит, взятый на образование и автомобиль, а чаще - на жилье </font> 

## Шаг 4. Общий вывод

<font color='blue'> Из анализа сделали следующие выводы: 
- самыми надежными клиентами являются бездетные клиенты, процент невозврата является 7%, в то время как семьи с 20 детьми не возвращают кредит в 10,5% случаев
- вдовы и вдовцы не возвращают кредит в срок лишь в 6,5% случаев, в то время как процент невозврата у холостых клиентов или клиентов в гражданском браке превышает 9%
- люди со средним доходом более склонны задержать выплату
- самые надежные клиенты это люди, берущие кредит на операции с недвижимостью (5-6% невозвратов в срок), а самые безответственные те, кто обращается в банк с целью получить деньги на покупку автомобиля или оплаты обучения (10-11%)</font> 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.